In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from fake_useragent import UserAgent

In [5]:
# 原網址：https://hwms.epa.gov.tw/dispPageBox/shop/shopCP.aspx?ddsPageID=TABLEWAREE&
#目前設定為只爬取 台北市 資料


#設定爬取結束頁數，目前全部42頁
end = 42


restaurants_list = []
for page in range(1, end+1):
    url = 'https://hwms.epa.gov.tw/dispPageBox/shop/shopCP.aspx?ddsPageID=TABLEWAREE'
    my_params = {'p': int(page)}

    # 設定header
    ua = UserAgent()
    user_agent = ua.random
    header = {'user-agent': user_agent}

    
    r = requests.get(url, params = my_params, headers=header)
    r.encoding= 'utf-8'
    html = r.content
    soup = BeautifulSoup(html,'html.parser')
    link_list= []
    href = "https://hwms.epa.gov.tw/"
    table = soup.find('table')
    for tr in table.find_all('tr'):
        for a in tr.find_all('a'):
            link = a.get('href')
            newlink = str(href) + str(link)
            link_list.append(newlink)
            
    # 若覺得有不必要attribute可以在這裡修改！
    not_choose_attribute = ["區域","地址"]

    
    for link in link_list:
        # 自訂header
        my_headers = {'user-agent': 'my-app/0.0.1'}
        r = requests.get(link, headers = my_headers)
        r.encoding= 'utf-8'
        html = r.content
        soup = BeautifulSoup(html,'html.parser')

        # 建立此餐廳的dic
        shop_dict = {}
        attribute_list = []
        value_list = []
        table = soup.find('tbody')
        for tr in table.find_all('tr'):
            attribute = tr.find('th').text.strip()
            value = tr.find('td').text.strip()


            # 選出台北地區餐廳(但不記錄區域這個attribute, 因會與地址重複)
            if attribute == "區域" and "台北" not in value and "臺北" not in value:
                break
            if attribute not in not_choose_attribute:
                shop_dict[attribute] = value
        if len(shop_dict) > 1:
            restaurants_list.append(shop_dict)

    #去除多餘空白換行
    for restaurant in restaurants_list:
        time = restaurant["起訖時間"]
        time = time.replace("\r\n","")
        time = time.replace(" ","")
        restaurant["起訖時間"] = time
        time = restaurant["營業時間"]
        time = time.replace("\r\n","")
        time = time.replace(" ","")
        restaurant["營業時間"] = time
        
        
# 寫檔
labels = ['餐廳名稱','餐廳類型','營業時間','電話','地址','好康類型','好康條件','好康內容','起訖時間']
with open('自備餐具優惠店家資料表.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames= [attr for attr in labels if attr not in not_choose_attribute])
        writer.writeheader()
        for elem in restaurants_list:
            writer.writerow(elem)

結束頁數: 10
